## Einleitung: Berechnung von Fondskennzahlen mit Python

### Hintergrund & Problemstellung

Im Unternehmen werden zurzeit viele Fondskennzahlen und Analysen in Excel und VBA erstellt. Bei größeren Datenmengen und komplexeren Anforderungen wie rollierenden Zeitreihenanalysen stößt diese Lösung jedoch an ihre Grenzen. Hintergrund dafür ist, dass nicht mehr einzelne Kennzahlen betrachtet werden, sondern zu jeder Kennzahl und jedem Zeitraum eine Zeitreihe für jeden Fonds vorliegen muss.

Die Analyse der rollierenden Kennzahlen soll als neue Auswertungsmethode implementiert werden, um zu erkennen, wie sich ein Fonds bezüglich einer bestimmten Kennzahl in unterschiedlichen Marktphasen entwickelt hat.

Darüber hinaus soll analysiert werden, wie sich ein Fonds im Vergleich zu Fonds aus seiner Peergroup über die Zeit entwickelt hat. In den Peergroups sind Fonds mit vergleichbarer Anlagestrategie gruppiert. Aus den rollierenden Kennzahlen soll ermittelt werden, ob ein Fonds über einen Zeitraum hinweg zu den besten seiner Peergroup zählte oder ob die Strategie im Zeitverlauf unterschiedlich erfolgreich war.

Des Weiteren ist jeder Peergroup eine Benchmark zugeordnet. Für diese Benchmarks sollen ebenfalls rollierende Kennzahlen berechnet werden, damit die Kennzahlen der Fonds mit der jeweiligen Benchmark verglichen werden können.

Zusätzlich werden Kennzahlen berechnet, die zeigen sollen, wie gut die Benchmark zur Anlagestrategie des Fonds passt. Dafür werden Benchmark-bezogene Kennzahlen wie Tracking Error, Beta und R² über die Zeit berechnet.

Außerdem sollen Korrelationen mit allgemeinen Marktbenchmarks aus unterschiedlichen Anlageklassen berechnet werden. Diese werden nicht rollierend berechnet und sollen einen Überblick darüber geben, wie sich ein Fonds im Verhältnis zu verschiedenen Anlageklassen verhält.

---

### Zielsetzung

Ziel ist es, eine skalierbare Python-Lösung zu entwickeln, die:

- alle relevanten Fondskennzahlen über rollierende Zeiträume (12, 24, 36, 60 Monate) berechnet,
- Perzentil-Ränge innerhalb der Peergroups erstellt, um die relative Fondsleistung zu bewerten,
- benchmarkbasierte Kennzahlen wie Tracking Error, Beta oder R² berechnet,
- und nicht-rollierende Korrelationen zu allgemeinen Benchmarks unterschiedlicher Anlageklassen liefert

Die berechneten Ergebnisse sollen sowohl in Pickle-Dateien (für andere Python-Programme) als auch in Excel-Dateien (für Mitarbeiter in der Analyse) gespeichert werden.

---

### Datenquellen

Für das Projekt werden die Daten in den Excel-Dateien `Returns.xlsx` und `Benchmarks.xlsx` bereitgestellt.

`Returns.xlsx` enthält die folgenden Fondsdaten:
- Fonds-Metadaten: Fondsname, ISIN, Peergroup, Ranking, FondsID
- Fonds-Zeitreihen: monatliche Renditen über 122 Monate von über 18.003 Fonds aus 222 Peergroups

`Benchmarks.xlsx` enthält:

- **Sheet "Ranking_Marktbenchmarks"**:
  - Benchmark-Zuordnung: Gibt die Peergroup zur jeweiligen Benchmark an
  - Benchmark-Zeitreihen: monatliche Renditen über 122 Monate von 204 Benchmarks (eine Benchmark kann mehreren Peergroups zugeordnet sein)

- **Sheet "Korrelationen"**:
  - Euribor-Zeitreihe: monatliche Renditen über 122 Monate (für die Berechnung der Sharpe Ratio)
  - Korrelation-Benchmarks: monatliche Renditen von 7 allgemeinen Benchmarks (z. B. Aktien, Renten, Rohstoffe), für die Korrelationen mit Fonds berechnet werden

---

### Vorgehen

Der Workflow gliedert sich in folgende Schritte:
1. Laden & Vorbereiten der Daten aus Excel-Dateien
2. Berechnung der rollierenden Kennzahlen für Fonds
3. Berechnung der Perzentil-Ränge in den Peergroups
4. Berechnung der rollierenden Kennzahlen für Benchmarks
5. Berechnung von Korrelationen mit allgemeinen Benchmarks
6. Speicherung der Ergebnisse als Pickle- und Excel-Dateien


## Ziel dieses Skripts

In diesem Notebook werden verschiedene Finanzkennzahlen für Investmentfonds berechnet. Der Fokus liegt auf rollierenden Zeitreihen**, um die Entwicklung von Kennzahlen über die Zeit zu analysieren. Zusätzlich werden **Vergleiche innerhalb von Peergroups** ermöglicht sowie die Passgenauigkeit von Benchmarks bewertet.

Da diese Art von Berechnungen mit Excel/VBA bei großen Datenmengen schnell an ihre Grenzen stößt, wird hier eine **performante Lösung mit Python und NumPy** umgesetzt.

Die Ergebnisse dienen als Grundlage für weitere Auswertungen (z. B. Visualisierung oder Machine Learning).


## Importieren der nötigen Packete und Kennzahlen aus Formeln.py 

In [ ]:
# Packete importieren
import numpy as np
import pandas as pd
import time
import os
import pickle
from pathlib import Path

# Kennzahlen aus Kennzahlberechnung.py importieren
from Formeln import annualized_return, volatility, maximum_drawdown
from Formeln import sharpe_ratio, tracking_error, beta, r_squared, up_correlation, down_correlation
from Formeln import omega_ratio, worst_month

## Daten laden und vorbereiten

In diesem Abschnitt werden die Rohdaten aus den Excel-Dateien `Returns.xlsx` und `Benchmarks.xlsx` geladen und aufbereitet, sodass die Kennzahlen mit NumPy  berechnet werden können.

`Returns.xlsx` enthält die monatlichen Fondsrenditen sowie die Metadaten der Fonds (Fondsname, ISIN, Peergroup, Ranking). Nach dem Laden werden:
- die benötigten Metadaten extrahiert, die für Analysen benötigt werdne können und daher mit den Ergebnissen gespeichert werden
- ein Mapping von Fonds zu Peergroups erstellt, das für die Zuordnung der Fonds zu den jeweiligen Peergroups benötigt wird
- die Renditezeitreihen in ein NumPy-Array (`rendite_array`) überführt, um diese für die Berechnung der rollierenden Kennzahlen zu verwenden.

`Benchmarks.xlsx` enthält die monatlichen Renditen für die Peergroup-Benchmarks, allgemeinen Korrelation-Benchmarks sowie die Zeitreihe des 3-Monats-Euribor. 

Die Peergroup-Benchmarks liegen im Blatt `"Ranking_Marktbenchmarks"` und beinhalten:
- die Renditezeitreihen der Peergroup-Benchmarks,
- sowie die Zuordnung von Peergroups zu ihrer jeweiligen Benchmark.

Die Benchmark-Zeitreihen werden in ein NumPy-Array (`benchmark_array`) überführt, das dieselbe Struktur wie die Fondsrenditen hat. Dadurch kann später bei jeder Berechnung direkt die passende Benchmark den jeweiligen Fonds zugeordnet werden.

Das Blatt `"Korrelationen"` beinhaltet:
- der 3-Monats-Euribor, der als risikoloser Zinssatz für die Sharpe-Ratio verwendet wird,
- sowie sieben allgemeine Benchmarks (z. B. Aktien- oder Rentenindizes), deren Korrelationen mit den Fondsrenditen berechnet werden.

Alle Zeitreihen werden einheitlich auf Monats-Datumsformate konvertiert. Die Benchmarks aus dem Korrelationen-Blatt werden ebenfalls als transponiertes NumPy-Array abgelegt (`benchmarks_korrelation_array`), damit sie für spätere Analysen effizient verwendet werden können.


In [109]:
# Fondsdaten laden
returns_df = pd.read_excel('./Daten/Returns.xlsx')
returns_df.drop(index=[0, 1, 2, 3, 4], inplace=True)
fonds_metadaten = returns_df[['FondsID', 'Fondsname', 'ISIN', 'Ranking', 'Peergroup']].drop_duplicates()
fonds_metadaten.set_index('FondsID', inplace=True)
returns_df.drop(['Ranking','Fondsname', 'ISIN'], axis=1, inplace=True)

# Mapping zwischen Fonds und Peergroup erstellen
fonds_zu_peergroup = dict(zip(returns_df['FondsID'], returns_df['Peergroup']))

# Daten für np.array aufbereiten
returns_df.drop(['Peergroup'], axis=1, inplace=True)
returns_df.set_index('FondsID', inplace=True)
returns_df.columns = pd.to_datetime(returns_df.columns, format='%Y-%m')
rendite_array = returns_df.astype(float).values.T

# fonds_ids und dates speichern
dates = returns_df.columns
fonds_ids = returns_df.index

# Benchmark-Daten laden
benchmarks = pd.read_excel('./Daten/Benchmarks.xlsx', sheet_name="Ranking_Marktbenchmarks")
benchmark_metadaten = benchmarks[['Benchmarkname', 'Peergroup', 'Ranking']].drop_duplicates()
benchmark_metadaten.set_index('Peergroup', inplace=True)

# Mapping zwischen Peergroups und Benchmarks erstellen
benchmark_mapping = benchmarks[['Peergroup', 'Benchmarkname']].drop_duplicates()
peergroup_zu_benchmark = dict(zip(benchmark_mapping['Peergroup'], benchmark_mapping['Benchmarkname']))

# Benchmark-Daten in die gleiche Struktur wie die Fondsrenditen aufbereiten
benchmarks.drop(['Ranking', 'Währung', 'Benchmarkname'], axis=1, inplace=True)
benchmarks.set_index('Peergroup',inplace=True)
benchmarks.columns = pd.to_datetime(benchmarks.columns, format='%Y-%m')

# np.array für Benchmarkrenditen erstellen, die der Struktur von rendite_array entspricht.
# Daduch kann die Benchmark-Rendite bei der Berechnung der Fondskennzahlen leichter den Fonds zugeordnet werden.
benchmark_array = np.full_like(rendite_array, np.nan)  # Gleiche Form wie rendite_array

# Benchmark-Renditen den Fonds zuordnen
for j, fonds_id in enumerate(returns_df.index):
    peergroup_name = fonds_zu_peergroup.get(fonds_id)
    benchmark_row = benchmarks.loc[peergroup_name]
    benchmark_values = benchmark_row.astype(float).values
    benchmark_array[:, j] = benchmark_values
    
# Euriborzeitreihe für die Berechnung der Sharpe Ratio laden
benchmarks_korrelation = pd.read_excel('./Daten/Benchmarks.xlsx', sheet_name="Korrelationen")
euribor_row = benchmarks_korrelation[benchmarks_korrelation['Benchmarkname'] == '3 Month Euribor']
euribor_array = euribor_row.drop('Benchmarkname', axis=1).astype(float).values[0]

# benchmarks für die Berechnung der Korrelationen laden
benchmarks_korrelation.set_index('Benchmarkname',inplace=True)
benchmarks_korrelation.drop('3 Month Euribor', axis=0, inplace=True)
benchmarks_korrelation.columns = pd.to_datetime(benchmarks_korrelation.columns, format='%Y-%m')
benchmarks_korrelation_namen = benchmarks_korrelation.index
benchmarks_korrelation_array = benchmarks_korrelation.astype(float).values.T

## Rollierende Berechnung der Fondskennzahlen

In diesem Abschnitt werden für jedes Zeitfenster (12, 24, 36 und 60 Monate) die Kennzahlen für alle Fonds berechnet.

Die Kennzahlen werden rollierend über die Zeit berechnet. Dadurch ergibt sich für jedes Zeitfenster eine Kennzahlenzeitreihe.

Es werden zwei Gruppen von Kennzahlen berechnet:
- Basis-Kennzahlen: Rendite, Volatilität, Maximum Drawdown, Worst Month, Omega-Ratio und Sharpe-Ratio
- Benchmark-Kennzahlen wie Tracking Error, Beta, Bestimmtheitsmaß (R2) und die Korrelation zur Peergroup-Benchmark.

Die Kennzahlen werden direkt mit NumPy auf den vorbereiteten Arrays berechnet, da sich dieser Weg als Performante Berechnungslösung herausstellte. Die Nutzung von `pandas.rolling()` wurde bewusst vermieden, da diese Methode bei großen Datensätzen langsamer ist und bei Kennzahlen mit zwei Zeitreihen wie den Benchmark-Kennzahlen 

Die Ergebnisse werden in einem Dictionary (`Ergebnisse`) gesammelt und in ein DataFrames umgewandelt.


In [ ]:
# Zeitfenster in Monaten
windows = [12, 24, 36, 60] 

# Dimensionen extrahieren
n_dates, n_fonds = rendite_array.shape

# Dictionary für Ergebnisse initialisieren
Ergebnisse= {}

# Berechnung der Kennzahlen für jedes Zeitfenster
for window in windows:
    
    # Arrays für die Kennzahlen initialisieren
    metrics = {
        f'Rendite_{window}M': np.full((n_dates, n_fonds), np.nan),
        f'Vola_{window}M': np.full((n_dates, n_fonds), np.nan),
        f'MaxDD_{window}M': np.full((n_dates, n_fonds), np.nan),
        f'WorstMonth_{window}M': np.full((n_dates, n_fonds), np.nan),
        f'Omega_{window}M': np.full((n_dates, n_fonds), np.nan),
        f'Sharpe_{window}M': np.full((n_dates, n_fonds), np.nan),
        f'TE_{window}M': np.full((n_dates, n_fonds), np.nan),
        f'Beta_{window}M': np.full((n_dates, n_fonds), np.nan),
        f'R2_{window}M': np.full((n_dates, n_fonds), np.nan),
        f'BmKorr_{window}M': np.full((n_dates, n_fonds), np.nan),
    }

    # Berechnung der Kennzahlen für jede Fonds
    for j in range(n_fonds):
        # Berechnung der Kennzahlen über die Zeiträume des Zeitfensters (i = Startzeitpunkt, end_idx = Endzeitpunkt)
        for i in range(n_dates - window + 1):
            end_idx = i + window
            
            # Zeitreihen für das aktuelle Zeitfenster extrahieren
            euribor_window = euribor_array[i:end_idx]
            returns_window = rendite_array[i:end_idx, j]
            
            # Prüft, dass keine NaN-Werte in den Renditen vorhanden sind
            if np.sum(np.isnan(returns_window)) == 0:
                # Berechnung der Basis-Kennzahlen
                metrics[f'Rendite_{window}M'][end_idx-1, j] = annualized_return(returns_window)
                metrics[f'Vola_{window}M'][end_idx-1, j] = volatility(returns_window)
                metrics[f'MaxDD_{window}M'][end_idx-1, j] = maximum_drawdown(returns_window)
                metrics[f'WorstMonth_{window}M'][end_idx-1, j] = worst_month(returns_window)
                metrics[f'Omega_{window}M'][end_idx-1, j] = omega_ratio(returns_window)
                
                # Berechnung der Sharpe Ratio mit Euribor
                # Prüft, dass keine NaN-Werte in den Euribor-Renditen vorhanden sind
                if np.sum(np.isnan(euribor_window)) == 0:
                    metrics[f'Sharpe_{window}M'][end_idx-1, j] = sharpe_ratio(returns_window, euribor_window)
                
                # Berechnung der Benchmark-Kennzahlen mit den Peergroup-Benchmarks
                benchmark_window = benchmark_array[i:end_idx, j]
                if np.sum(np.isnan(benchmark_window)) == 0:
                    metrics[f'TE_{window}M'][end_idx-1, j] = tracking_error(returns_window, benchmark_window)
                    metrics[f'Beta_{window}M'][end_idx-1, j] = beta(returns_window, benchmark_window)
                    metrics[f'R2_{window}M'][end_idx-1, j] = r_squared(returns_window, benchmark_window)
                    metrics[f'BmKorr_{window}M'][end_idx-1, j] = np.corrcoef(returns_window, benchmark_window)[0, 1]
        
    # NumPy Arrays in DataFrames umwandeln
    for metric_name, metric_array in metrics.items():
        Ergebnisse[metric_name] = pd.DataFrame(
            data=metric_array,
            index=dates,           
            columns=fonds_ids
        )


## Berechnung der Perzentil-Ränge in der Peergroup

In diesem Abschnitt werden für die Zeitfenster (12, 24, 36 und 60 Monate) die Zeitreihen der Perzentilränge der Fonds Berechnet. Dafür wird für jede Basis-Kennzahl und jeden Zeitpunkt eines Zeitfensters der Perzentil-Rang des Fonds im Vergleich zu den anderen Fonds derselben Peergroup berechnet.

Die Berechnung erfolgt auf Basis der bereits zuvor berechneten rollierenden Kennzahlen. Für jeden Zeitpunkt wird innerhalb jeder Peergroup eine Perzentil-Rangliste erstellt. Diese berücksichtigt nur die Fonds, die zu diesem Zeitpunkt einen Kennzahlwert aufweisen. 

Die Ergebnisse werden in einem Dictionary (`Percentil_Ergebnisse`) gesammelt und in ein DataFrames umgewandelt.

In [ ]:
# Definieren der Basis-Kennzahlen für die der Perzentil-Rang berechnet wird
percentil_kennzahlen = ['Rendite', 'Vola', 'MaxDD', 'WorstMonth', 'Omega', 'Sharpe']

# Dictionary für Ergebnisse initialisieren
Percentil_Ergebnisse = {}

# Liste an Fonds_IDs und Dictionary mit zuordnung der FondsID zur Peergroup erstellen
fonds_ids = list(fonds_metadaten.index)
peergroups = fonds_metadaten['Peergroup'].to_dict()

# Berechnung der Perzentil-Ränge für die Basis-Kennzahlen
for kpi in percentil_kennzahlen:
    # Berechnung der Perzentil-Ränge für jedes Zeitfenster
    for window in windows:
        # Arrays mit den benötigten Kennzahlen extrahieren
        df = Ergebnisse[f"{kpi}_{window}M"]
        df_array = df.to_numpy()
        ranks_array = np.full_like(df_array, np.nan)

        # Berechnung der Perzentil-Ränge über die Zeiträume des Zeitfensters
        for t in range(df_array.shape[0]):
            # Kennzahlwerte für den aktuellen Zeitpunkt extrahieren
            row = df_array[t, :]
            # Berechnung der Perzentil-Ränge für jede Peergroup
            for pg in set(peergroups.values()):
                # Ermitteln der Fonds in der aktuellen Peergroup und extrahieren der Fonds Indizes
                fonds_in_pg = [i for i, f in enumerate(fonds_ids) if peergroups.get(f) == pg]
                # Extrahieren der Kennzahlwerte für die Fonds in der Peergroup
                values = row[fonds_in_pg]
                # Prüfen, ob für den Zeitpunkt keine Kennzahlenwerte vorhanden sind
                if np.isnan(values).all():
                    continue
                # Berechnung der Perzentil-Ränge für die Fonds in der Peergroup und Zuordnug der Werte
                ranks = pd.Series(values).rank(pct=True).values
                for idx, i in enumerate(fonds_in_pg):
                    ranks_array[t, i] = ranks[idx]

         # Ergebnisse von NumPy Arrays in DataFrames umwandeln
        Percentil_Ergebnisse[f"{kpi}_{window}M"] = pd.DataFrame(
            data=ranks_array,
            index=df.index,
            columns=df.columns
        )

## Berechnung der Kennzahlen für Peergroup-Benchmarks

In diesem Abschnitt werden ür jedes Zeitfenster (12, 24, 36 und 60 Monate) die Basis-Kennzahlen (Rendite, Volatilität, Maximum Drawdown, Worst Month, Omega-Ratio und Sharpe-Ratio) für die Peergroup-Benchmarks berechnet. 

Die Berechnung erfolgt annalog zur Berechnung der Basis-Kennzahlen für die Fonds.

Die Ergebnisse werden in einem Dictionary (`Benchmark_Ergebnisse`) gesammelt und in ein DataFrames umgewandelt.

In [114]:
# Dimensionen und Benchmarknamen extrahieren
benchmark_ids = benchmarks.index
n_benchmarks = len(benchmark_ids)

# Benchmark-Renditen in ein transponiertes NumPy-Array umwandeln
benchmark_array_all = benchmarks.astype(float).values.T

# Dictionary für die Ergebnisse initialisieren
Benchmark_Ergebnisse = {}

# Berechnung der Benchmark-Kennzahlen für jedes Zeitfenster
for window in windows:
    # Arrays für die Kennzahlen initialisieren
    metrics = {
        f'Rendite_{window}M': np.full((n_dates, n_benchmarks), np.nan),
        f'Vola_{window}M': np.full((n_dates, n_benchmarks), np.nan),
        f'MaxDD_{window}M': np.full((n_dates, n_benchmarks), np.nan),
        f'WorstMonth_{window}M': np.full((n_dates, n_benchmarks), np.nan),
        f'Omega_{window}M': np.full((n_dates, n_benchmarks), np.nan),
        f'Sharpe_{window}M': np.full((n_dates, n_benchmarks), np.nan),
    }

    # Berechnung der Benchmark-Kennzahlen für alle Benchmarks
    for j in range(n_benchmarks):
        # Berechnung der Benchmark-Kennzahlen über die Zeiträume des Zeitfensters (i = Startzeitpunkt, end_idx = Endzeitpunkt)
        for i in range(n_dates - window + 1):
            end_idx = i + window
            
            # Zeitreihen für das aktuelle Zeitfenster extrahieren
            returns_window = benchmark_array_all[i:end_idx, j]
            euribor_window = euribor_array[i:end_idx]

            # Prüft, dass keine NaN-Werte in den Renditen vorhanden sind
            if np.sum(np.isnan(returns_window)) == 0:
                # Berechnung der Basis-Kennzahlen
                metrics[f'Rendite_{window}M'][end_idx-1, j] = annualized_return(returns_window)
                metrics[f'Vola_{window}M'][end_idx-1, j] = volatility(returns_window)
                metrics[f'MaxDD_{window}M'][end_idx-1, j] = maximum_drawdown(returns_window)
                metrics[f'WorstMonth_{window}M'][end_idx-1, j] = worst_month(returns_window)
                metrics[f'Omega_{window}M'][end_idx-1, j] = omega_ratio(returns_window)
                # Berechnung der Sharpe Ratio mit Euribor
                # Prüft, dass keine NaN-Werte in den Euribor-Renditen vorhanden sind
                if np.sum(np.isnan(euribor_window)) == 0:
                    metrics[f'Sharpe_{window}M'][end_idx-1, j] = sharpe_ratio(returns_window, euribor_window)
                    
    # Ergebnisse von NumPy Arrays in DataFrames umwandeln
    for metric_name, metric_array in metrics.items():
        Benchmark_Ergebnisse[metric_name] = pd.DataFrame(
            data=metric_array,
            index=dates,
            columns=benchmark_ids
        )

## Berechnung der Korrelationen mit allgemeinen Benchmarks

In diesem Abschnitt werden für jeden Fonds die Korrelationen zu sieben allgemeinen Benchmarks berechnet. Dabei handelt es sich um Benchmarks unterschiedlicher Asset-Klassen (Corporate Bonds, Aktien Welt, Bund, High Yield, Brent, Gold, Vix). Die Korrelationen sollen einen einblick geben wie der Fonds sich gegenüber unterschiedlichen Anlageklassen verhält. Diese Werte dienen später u. a. als Input für die Machine-Learning-Modelle, um das Verhalten eines Fonds in Relation zu verschiedenen Anlageklassen zu charakterisieren.

Die Berechnung erfolgt nicht rollierend über alle Zeitpunkte, sondern nur für ein festes Zeitfenster am Ende der Zeitreihe. Berechnet werden die Korrelationen für die Zeiträume 12, 24, 36 oder 60 Monate.

Für jede Kombination aus Fonds und Benchmark werden drei verschiedene Korrelationen berechnet:
- Gesamtkorrelation
- Up-Korrelation (nur in Zeitpunkten, in denen die Benchmark positiv Renditen aufweist)
- Down-Korrelation (nur in Zeitpunkten, in denen die Benchmark negative Renditen aufweist)

Die Ergebnisse werden in einem Dictionary (`Korr_Ergebnisse`) gesammelt und in ein DataFrames umgewandelt.

In [115]:
# Zeitfenster in Monaten
windows = [12, 24, 36, 60]

# Dimensionen
n_dates, n_korrelation = benchmarks_korrelation_array.shape

# Dictionary für Ergebnisse
Korr_Ergebnisse = {}

# Berechnung der Kennzahlen für jedes Zeitfenster
for window in windows:
    # Arrays für die Kennzahlen initialisieren
    korr_metrics = {
        f'Korr_{window}M': np.full((n_korrelation, n_fonds), np.nan),
        f'UpKorr_{window}M': np.full((n_korrelation, n_fonds), np.nan),
        f'DownKorr_{window}M': np.full((n_korrelation, n_fonds), np.nan),
    }
    # Berechnung der Kennzahlen für jeden Fonds
    for j in range(n_fonds):
        # Berechnung der Kennzahlen über alle allgemeinen Benchmarks. Nicht rollierend berechnet
        for i in range(n_korrelation):

            # Zeitreihe für den Berechnungszeitraum extrahieren
            start_idx = n_dates - window + 1
            end_idx = n_dates
            korrelation_window = benchmarks_korrelation_array[start_idx:end_idx, i]
            returns_window = rendite_array[start_idx:end_idx, j]

            # Prüft, dass keine NaN-Werte in den Fondsrenditen und Benchmarkrenditen vorhanden sind
            if np.sum(np.isnan(returns_window)) == 0 and np.sum(np.isnan(korrelation_window)) == 0:
                # Berechnung der unterschiedlichen Korrelationen
                korr_metrics[f'Korr_{window}M'][i, j] = np.corrcoef(returns_window, korrelation_window)[0, 1]
                korr_metrics[f'UpKorr_{window}M'][i, j] = up_correlation(returns_window, korrelation_window)
                korr_metrics[f'DownKorr_{window}M'][i, j] = down_correlation(returns_window, korrelation_window)

    # Ergebnisse von NumPy Arrays in DataFrames umwandeln
    for Korr_metric_name, Korr_metric_array in korr_metrics.items():
        Korr_Ergebnisse[Korr_metric_name] = pd.DataFrame(
            data=Korr_metric_array,
            index=benchmarks_korrelation_namen,           
            columns=fonds_ids
        )

    

## Speichern der Ergebnisse

In Diesem Abschnitt werden alle berechneten Kennzahlen und Metadaten in Pickle- und Excel-Dateien gespeichert. Dabei dienen Pickle-Dateien dazu, dass die anderen Python-Skripten weiterverarbeitet die berechneten Kennzahlen schnell laden können. Die Excel-Dateien dienen dazu, dass die Dateien von Mitarbeitern für analysen verwendet werden können. 

Die Ergebnisse werden nach Kennzahl und Zeitfenster sortiert und in Dictionaries organisiert. Diese werden jeweils als `.pkl` und `.xlsx` gespeichert. Auch die Perzentil-Ränge, Benchmark-Kennzahlen und Korrelationen werden analog gespeichert. Zusätzlich werden die Fonds- und Benchmark-Metadaten exportiert.


In [120]:
ordner = './Ergebnisse'
os.makedirs(ordner, exist_ok=True)

# Kennzahlen nach Typ gruppieren
kennzahl_typen = {}
for name, df in Ergebnisse.items():
    typ, window = name.split('_')
    kennzahl_typen.setdefault(typ, {})[window] = df

for typ, windows_str in kennzahl_typen.items():
    with open(f'{ordner}/{typ}.pkl', 'wb') as f:
        pickle.dump(windows_str, f)
    with pd.ExcelWriter(f'{ordner}/{typ}.xlsx', engine='openpyxl') as writer:
        for window, df in windows_str.items():
            df.T.to_excel(writer, sheet_name=window)
    print(f"{typ} gespeichert (Pickle & Excel)")

# Benchmark-Kennzahlen speichern
benchmark_typen = {}
for name, df in Benchmark_Ergebnisse.items():
    typ, window = name.split('_')
    benchmark_typen.setdefault(typ, {})[window] = df

for typ, windows_str in benchmark_typen.items():
    with open(f'{ordner}/Benchmarks_{typ}.pkl', 'wb') as f:
        pickle.dump(windows_str, f)
    with pd.ExcelWriter(f'{ordner}/Benchmarks_{typ}.xlsx', engine='openpyxl') as writer:
        for window, df in windows_str.items():
            df.T.to_excel(writer, sheet_name=window)
    print(f"Benchmark-{typ} gespeichert (Pickle & Excel)")

# Percentil-Ränge speichern
percentil_typen = {}
for name, df in Percentil_Ergebnisse.items():
    typ, window = name.split('_')
    percentil_typen.setdefault(typ, {})[window] = df

for typ, windows_str in percentil_typen.items():
    with open(f'{ordner}/Percentil_{typ}.pkl', 'wb') as f:
        pickle.dump(windows_str, f)
    with pd.ExcelWriter(f'{ordner}/Percentil_{typ}.xlsx', engine='openpyxl') as writer:
        for window, df in windows_str.items():
            df.T.to_excel(writer, sheet_name=window)
    print(f"Percentil-{typ} gespeichert (Pickle & Excel)")

# Korrelationsergebnisse speichern
if Korr_Ergebnisse:
    with open(f'{ordner}/Korrelationen.pkl', 'wb') as f:
        pickle.dump(Korr_Ergebnisse, f)

    with pd.ExcelWriter(f'{ordner}/Korrelationen.xlsx', engine='openpyxl') as writer:
        for name, df in Korr_Ergebnisse.items():
            df.T.to_excel(writer, sheet_name=name)

    print("Korrelationen gespeichert (Pickle & Excel)")

# Fonds-Metadaten speichern
os.makedirs('./Ergebnisse', exist_ok=True)
fonds_metadaten.to_pickle('./Ergebnisse/fonds_metadata.pkl')
with pd.ExcelWriter(f'{ordner}/fonds_metadata.xlsx', engine='openpyxl') as writer:
    fonds_metadaten.to_excel(writer, sheet_name=name)
print("Fonds-Metadaten gespeichert")

# Benchmark-Metadaten speichern
os.makedirs('./Ergebnisse', exist_ok=True)
benchmark_metadaten.to_pickle('./Ergebnisse/benchmark_metadata.pkl')
with pd.ExcelWriter(f'{ordner}/benchmark_metadata.xlsx', engine='openpyxl') as writer:
    benchmark_metadaten.to_excel(writer, sheet_name=name)
print("Benchmark-Metadaten gespeichert")

Rendite gespeichert (Pickle & Excel)
Vola gespeichert (Pickle & Excel)
MaxDD gespeichert (Pickle & Excel)
WorstMonth gespeichert (Pickle & Excel)
Omega gespeichert (Pickle & Excel)
Sharpe gespeichert (Pickle & Excel)
TE gespeichert (Pickle & Excel)
Beta gespeichert (Pickle & Excel)
R2 gespeichert (Pickle & Excel)
BmKorr gespeichert (Pickle & Excel)
Benchmark-Rendite gespeichert (Pickle & Excel)
Benchmark-Vola gespeichert (Pickle & Excel)
Benchmark-MaxDD gespeichert (Pickle & Excel)
Benchmark-WorstMonth gespeichert (Pickle & Excel)
Benchmark-Omega gespeichert (Pickle & Excel)
Benchmark-Sharpe gespeichert (Pickle & Excel)
Percentil-Rendite gespeichert (Pickle & Excel)
Percentil-Vola gespeichert (Pickle & Excel)
Percentil-MaxDD gespeichert (Pickle & Excel)
Percentil-WorstMonth gespeichert (Pickle & Excel)
Percentil-Omega gespeichert (Pickle & Excel)
Percentil-Sharpe gespeichert (Pickle & Excel)
Korrelationen gespeichert (Pickle & Excel)
Fonds-Metadaten gespeichert
Benchmark-Metadaten gesp

## Fazit: Kennzahlenberechnung mit Python

Die Berechnung der Kennzahlen konnte erfolgreich und performant in Python umgesetzt werden. Durch die Nutzung von NumPy und gezielter Datenstrukturierung ist die schnelle Berechnung und Speicherung großer Datenmengen möglich.

Die Ergebnisse lassen sich strukturiert in Pickle- und Excel-Dateien speichern und stehen so sowohl für die Weiterverarbeitung in Python – wie beispielhaft im Dashboard (`Dashboard.py`) dargestellt – als auch für manuelle Analysen im Unternehmen zur Verfügung.

Daher kann die Implementierung zukünftig für die Berechnung und für Analysen eingesetzt werden.
